# Vector Database Tutorial

## Install & import the libraries required - Will be using LangChain

In [60]:
!pip install openai numpy pandas singlestoredb langchain==0.1.8 langchain-community==0.0.21 langchain-core==0.1.25 langchain-openai==0.0.6 --quiet

In [61]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.singlestoredb import SingleStoreDB

In [62]:
import openai
import os
import pandas as pd
import requests

## Mention OpenAI API key

In [67]:
import os
os.environ['OPENAI_API_KEY'] = 'mention your OpenAI API key'

## Build a simple chatbot

Will download some publicly available story, conert the content into embeddings and store it in our vector database and start chatting.

In [ ]:
# First, download the text file content from the URL
file_url = "https://sherlock-holm.es/stories/plain-text/stud.txt"
response = requests.get(file_url)
if response.status_code == 200:
    file_content = response.text
    file_path = 'local_example.txt'  # Save the content to a local file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(file_content)
else:
    raise Exception("Failed to download the file.")

# Ensure the OPENAI_API_KEY is set as an environment variable
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_KEY:
    raise Exception("OPENAI_API_KEY environment variable not set. Please set it before running the script.")

# Initialize the OpenAI client with the API key
client = openai.OpenAI(api_key=OPENAI_KEY)

# Load and process documents from the local file
loader = TextLoader(file_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Initialize embeddings and create a document search database
embeddings = OpenAIEmbeddings()
docsearch = SingleStoreDB.from_documents(docs, embeddings, table_name="sample")

# Chat loop
while True:
    user_query = input("\nYou: ")
    if user_query.lower() in ['quit', 'exit']:
        print("Exiting chatbot.")
        break

    docs = docsearch.similarity_search(user_query)
    if docs:
        context = docs[0].page_content
        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=f"Context: {context}\n\nUser: {user_query}\nAI:",
            temperature=0.7,
            max_tokens=150,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        print("AI:", response.choices[0].text.strip())
    else:
        print("AI: Sorry, I couldn't find relevant information.")


You:  which university did they get the degree from?


AI: There is no mention of a university in the text. The immigrants were Mormons, who were led by their leader, presumably Brigham Young, to settle in the promised land of Utah. It is unlikely that they received any formal education or degrees during their journey and settlement.



You:  what the story is all about?


AI: The story is about an old farmer who is facing a crisis in a settlement surrounded by mountains. The Council has issued an order that all residents must leave or face dire consequences. The farmer's daughter is in danger, and the old man is struggling to find a way out of the situation. As the days pass and the numbers of residents dwindle, he becomes more and more desperate. He is determined to protect his daughter at all costs, but it seems that there is no escape from the Council's control. The story explores themes of power, sacrifice, and the human desire to protect the ones we love.



You:  who is watson in the story?


AI: Dr. Watson is a character in the story, who is a close friend and assistant to the famous detective Sherlock Holmes. He is a retired army doctor and serves as the narrator of the story.



You:  who is stamford?


AI: Stamford is a friend of the narrator, who used to serve as a dresser under him at Bart's.
